In [1]:
# https://towardsdatascience.com/a-simple-guide-to-the-versions-of-the-inception-network-7fc52b863202

In [2]:
import  os
import  tensorflow as tf
import  numpy as np
from    tensorflow import keras

2021-09-18 04:30:07.003574: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


In [3]:
tf.random.set_seed(22)
np.random.seed(22)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
assert tf.__version__.startswith('2.')

In [4]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train, x_test = x_train.astype(np.float32)/255., x_test.astype(np.float32)/255.
# [b, 28, 28] => [b, 28, 28, 1]
x_train, x_test = np.expand_dims(x_train, axis=3), np.expand_dims(x_test, axis=3)

db_train = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(256)
db_test = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(256)

11493376/11490434 [==============================] - 0s 0us/step


2021-09-18 04:30:35.488469: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-09-18 04:30:35.491909: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-09-18 04:30:35.547139: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-09-18 04:30:35.547767: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla P100-PCIE-16GB computeCapability: 6.0
coreClock: 1.3285GHz coreCount: 56 deviceMemorySize: 15.90GiB deviceMemoryBandwidth: 681.88GiB/s
2021-09-18 04:30:35.547822: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2021-09-18 04:30:35.575806: I tensorflow/stream_executor/platform/def

In [5]:
print(x_train.shape, type(x_train))

(60000, 28, 28, 1) <class 'numpy.ndarray'>


In [6]:
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(60000, 28, 28, 1) (60000,)
(10000, 28, 28, 1) (10000,)


In [7]:
class ConvBNRelu(keras.Model):
    
    def __init__(self, ch, kernelsz=3, strides=1, padding='same'):
        super(ConvBNRelu, self).__init__()
        
        self.model = keras.models.Sequential([
            keras.layers.Conv2D(ch, kernelsz, strides=strides, padding=padding),
            keras.layers.BatchNormalization(),
            keras.layers.ReLU()
        ])
        
        
    def call(self, x, training=None):
        
        x = self.model(x, training=training)
        
        return x 

In [8]:
class InceptionBlk(keras.Model):
    
    def __init__(self, ch, strides=1):
        super(InceptionBlk, self).__init__()
        
        self.ch = ch
        self.strides = strides

        self.conv1 = ConvBNRelu(ch, strides=strides)
        self.conv2 = ConvBNRelu(ch, kernelsz=3, strides=strides)
        self.conv3_1 = ConvBNRelu(ch, kernelsz=3, strides=strides)
        self.conv3_2 = ConvBNRelu(ch, kernelsz=3, strides=1)
        
        self.pool = keras.layers.MaxPooling2D(3, strides=1, padding='same')
        self.pool_conv = ConvBNRelu(ch, strides=strides)
        
        
    def call(self, x, training=None):
        
        
        x1 = self.conv1(x, training=training)

        x2 = self.conv2(x, training=training)
                
        x3_1 = self.conv3_1(x, training=training)
        x3_2 = self.conv3_2(x3_1, training=training)
                
        x4 = self.pool(x)
        x4 = self.pool_conv(x4, training=training)
        
        # concat along axis=channel
        x = tf.concat([x1, x2, x3_2, x4], axis=3)
        
        return x

In [9]:
class Inception(keras.Model):
    
    def __init__(self, num_layers, num_classes, init_ch=16, **kwargs):
        super(Inception, self).__init__(**kwargs)
        
        self.in_channels = init_ch
        self.out_channels = init_ch
        self.num_layers = num_layers
        self.init_ch = init_ch
        
        self.conv1 = ConvBNRelu(init_ch)
        
        self.blocks = keras.models.Sequential(name='dynamic-blocks')
        
        for block_id in range(num_layers):
            
            for layer_id in range(2):
                
                if layer_id == 0:
                    
                    block = InceptionBlk(self.out_channels, strides=2)
                    
                else:
                    block = InceptionBlk(self.out_channels, strides=1)
                    
                self.blocks.add(block)
            
            # enlarger out_channels per block    
            self.out_channels *= 2
            
        self.avg_pool = keras.layers.GlobalAveragePooling2D()
        self.fc = keras.layers.Dense(num_classes)
        
        
    def call(self, x, training=None):
        
        out = self.conv1(x, training=training)
        
        out = self.blocks(out, training=training)
        
        out = self.avg_pool(out)
        out = self.fc(out)
        
        return out    

In [10]:
# build model and optimizer
batch_size = 32
epochs = 100
model = Inception(2, 10)
# derive input shape for every layers.
model.build(input_shape=(None, 28, 28, 1))
model.summary()

optimizer = keras.optimizers.Adam(learning_rate=1e-3)
criteon = keras.losses.CategoricalCrossentropy(from_logits=True)

acc_meter = keras.metrics.Accuracy()

Model: "inception"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_bn_relu (ConvBNRelu)    multiple                  224       
_________________________________________________________________
dynamic-blocks (Sequential)  (None, 7, 7, 128)         292704    
_________________________________________________________________
global_average_pooling2d (Gl multiple                  0         
_________________________________________________________________
dense (Dense)                multiple                  1290      
Total params: 294,218
Trainable params: 293,226
Non-trainable params: 992
_________________________________________________________________


In [11]:
for epoch in range(100):

    for step, (x, y) in enumerate(db_train):

        with tf.GradientTape() as tape:
            # print(x.shape, y.shape)
            # [b, 10]
            logits = model(x)
            # [b] vs [b, 10]
            loss = criteon(tf.one_hot(y, depth=10), logits)

        grads = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))

        if step % 10 == 0:
            print(epoch, step, 'loss:', loss.numpy())


    acc_meter.reset_states()
    for x, y in db_test:
        # [b, 10]
        logits = model(x, training=False)
        # [b, 10] => [b]
        pred = tf.argmax(logits, axis=1)
        # [b] vs [b, 10]
        acc_meter.update_state(y, pred)

    print(epoch, 'evaluation acc:', acc_meter.result().numpy())

2021-09-18 04:38:50.963871: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.8
2021-09-18 04:38:56.117363: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.11
2021-09-18 04:38:56.848805: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublasLt.so.11


0 0 loss: 2.3067927
0 10 loss: 2.1030269
0 20 loss: 1.5699424
0 30 loss: 1.3794844
0 40 loss: 0.9233192
0 50 loss: 0.8301166
0 60 loss: 0.5781583
0 70 loss: 0.35383868
0 80 loss: 0.56293434
0 90 loss: 0.4557651
0 100 loss: 0.39482278
0 110 loss: 0.3665469
0 120 loss: 0.33006558
0 130 loss: 0.20413844
0 140 loss: 0.2431204
0 150 loss: 0.21798
0 160 loss: 0.23220327
0 170 loss: 0.18612988
0 180 loss: 0.30192673
0 190 loss: 0.13770968
0 200 loss: 0.2680411
0 210 loss: 0.17704089
0 220 loss: 0.16800207
0 230 loss: 0.036778033
0 evaluation acc: 0.9539
1 0 loss: 0.17814125
1 10 loss: 0.15480125
1 20 loss: 0.2226929
1 30 loss: 0.26450413
1 40 loss: 0.34233713
1 50 loss: 0.2944184
1 60 loss: 0.22319412
1 70 loss: 0.12525977
1 80 loss: 0.14425951
1 90 loss: 0.12286508
1 100 loss: 0.16167735
1 110 loss: 0.21939492
1 120 loss: 0.119434066
1 130 loss: 0.12410558
1 140 loss: 0.13017684
1 150 loss: 0.10345536
1 160 loss: 0.1109103
1 170 loss: 0.08285951
1 180 loss: 0.16340841
1 190 loss: 0.06397399
